In [11]:
import torch
from torch import nn
import torch.nn.functional as F
from torchvision.models import resnet18, resnet34, resnet50, resnet101, resnet152

from models.resnet_real import ResNet18, ResNet34, ResNet50, ResNet101, ResNet152

from data_loaders.imagenet import Train, Val
import numpy as np
from tqdm import tqdm, trange
from sklearn.metrics import accuracy_score

Speed in it/s
| Model | TorchVision | Ours |
|:--:|:--:|:--:|
| ResNet-18 | 50 | 25 |
| ResNet-34 | 34 | 16 |
| ResNet-50 | 16.5 | 16 |
| ResNet-101 | 9.7 | 9.5 |
| ResNet-152 | 6.8 | 6.7 |

In [12]:
# model = resnet152()
model = ResNet152()

hparams = {
    "batch_size": 256,
    "num_epochs": 40,
    "model": "ResNet18",
    "dataset": "imagenet",
    "num_classes": 20,
    "optimizer": "sgd",
    "learning_rate": 0.05,
    "device": "cuda:0",
}

GPU = torch.device(hparams["device"])
CPU = torch.device('cpu')

model.to(GPU)
optimiser = torch.optim.SGD(model.parameters(), lr=hparams["learning_rate"])
loss_fn = nn.CrossEntropyLoss()

In [13]:
# Loading data
training_generator = torch.utils.data.DataLoader(Train(d4=False), batch_size=hparams["batch_size"], num_workers=4)
validation_generator = torch.utils.data.DataLoader(Val(d4=False), batch_size=450, num_workers=4)
# accuracies

In [14]:
batch_size = hparams["batch_size"]
num_epochs = hparams["num_epochs"]

train_acc, test_acc = [], []

for epoch in range(num_epochs):
    print("Training")
    for batch_x, batch_y in tqdm(training_generator):
        # print("Batching")
        # print(f"{batch_x.shape = }, {batch_y.shape = }")
        # print(batch_y.max(), batch_y.min())
        batch_x, batch_y = batch_x.to(GPU), F.one_hot(batch_y.long().flatten(), num_classes=1000).to(GPU)
        optimiser.zero_grad()
        output = model(batch_x)
        # print(f"{output.shape = }, {batch_y.shape = }")
        loss = loss_fn(output, batch_y.argmax(1))
        loss.backward()
        optimiser.step()
    # losses.append(loss.item())
    

    print("Calculating Accuracy")

    train_accs, test_accs = [], []

    j = 0
    for batch_x, batch_y in training_generator:
        batch_x, batch_y = batch_x.to(GPU), batch_y.flatten()
        train_pred = model(batch_x)
        acc = accuracy_score(batch_y.numpy(), train_pred.argmax(1).cpu().numpy())
        train_accs.append(acc*100)
        j += 1
        if j >= 100: break
    train_acc.append(np.array(train_accs).mean())
    print(f"Train Accuracy: {train_acc[-1]:.2f}%")


    j = 0
    for batch_x, batch_y in validation_generator:
        batch_x, batch_y = batch_x.to(GPU), batch_y.flatten()
        test_pred = model(batch_x)
        acc = accuracy_score(batch_y.numpy(), test_pred.argmax(1).cpu().numpy())
        train_accs.append(acc*100)
        j += 1
        if j >= 100: break
    test_acc.append(np.array(train_accs).mean())
    print(f"Test Accuracy: {test_acc[-1]:.2f}%")

Training


  2%|▏         | 118/5005 [00:18<12:31,  6.51it/s]


KeyboardInterrupt: 